In [1]:
from web3 import Web3, HTTPProvider
import random 

random.seed(10)

contract_manager = Web3.to_checksum_address(0xcA489A0E111a33d7d3244108Fe0fCf5e9F351772)

In [2]:
# Connect to the chain

from web3.middleware import geth_poa_middleware

w3 = Web3(HTTPProvider('http://localhost:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.default_account = contract_manager

In [3]:
# New a DecentralElect Contract in the chain

from DecentralElect import DecentralElectContract

decentral_elect = DecentralElectContract(w3)
tx_rcpt = decentral_elect.deploy(contract_manager)
print('Contract Address:', tx_rcpt.contractAddress)

Contract Address: 0xC05FF86A5CFeba0BC5328C18e17EBcBcFBA3Af53


In [4]:
# Initilize the vote
# New the vote, and add the subject and candidates to it.

election_list =[
    {
        'id': 0, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 1, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 2, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 3, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
    {
        'id': 4, 
        'subject': 'What fruit do you like?',
        'candidates': [
            {'name': 'Apple',
             'description': 'Red'}, 
            {'name': 'Grape',
             'description': 'Purple'}, 
            {'name': 'Orange',
             'description': 'Orange'}, 
            {'name': 'Guava',
             'description': 'Green'}, 
            {'name': 'Banana',
             'description': 'Yellow'}
            ]
    },
]

for election in election_list:
    print('Status:', decentral_elect.get_status(election['id']))
    tx_rcpt = decentral_elect.create_election(identifier = election['id'], subject = election['subject'])
    print('Status:', decentral_elect.get_status(election['id']))

    for candidate in election['candidates']:
        decentral_elect.set_candidate(identifier = election['id'], name = candidate['name'], description = candidate['description'])

    tx_rcpt = decentral_elect.get_candidates(election['id'])
    print(tx_rcpt)


Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]
Status: Unopened
Status: Registration
[(0, 'Apple', 'Red', 0), (1, 'Grape', 'Purple', 0), (2, 'Orange', 'Orange', 0), (3, 'Guava', 'Green', 0), (4, 'Banana', 'Yellow', 0)]


In [5]:
from DecentralElect import Voter

register_list = [
    {
        'id': 0,
        'registers' : [Voter() for _ in range(10)]
    },
    # {
    #     'id': 1,
    #     'registers' : [Voter() for _ in range(20)]
    # },
    # {
    #     'id': 2,
    #     'registers' : [Voter() for _ in range(30)]
    # },
    # {
    #     'id': 3,
    #     'registers' : [Voter() for _ in range(40)]
    # },
    # {
    #     'id': 4,
    #     'registers' : [Voter() for _ in range(50)]
    # },
]

for election in register_list:
    for register in election['registers']:
        # print(decentral_elect.check_if_registered(election['id'], register.public_key[0].n))
        tx_rcpt = decentral_elect.register_voter(election['id'], register.public_key[0].n, register.public_key[1].n)
        if not decentral_elect.check_if_registered(election['id'], register.public_key[0].n):
            print('Error', register)
            

    # tx_rcpt = decentral_elect.get_registered_pkeys(election['id'])
    # print('Election', election['id'], ':', len(tx_rcpt), 'registed')
    # print(tx_rcpt)
            

In [6]:
from DecentralElect.uaosring import uaosring_sign
from DecentralElect import hash_vote
from DecentralElect.utils import bytes_to_int, pkeys_type_transform
from DecentralElect import LinkableRingSignature
import random 
import string

def generate_random_strings(num_strings:int, leading_numbers:list[int]):
    random_strings = []

    for _ in range(num_strings):
        leading_number = random.choice(leading_numbers)
        
        rest_of_string = ''.join(random.choices(string.ascii_letters + string.digits, k=20))
    
        random_string = str(leading_number) + rest_of_string
        random_strings.append(random_string)

    return random_strings

voting_list = register_list
voting_list[0]['vote'] = generate_random_strings(10, [0 ,1, 2, 3, 4])
# voting_list[1]['vote'] = generate_random_strings(20, [0 ,1, 2, 3, 4])
# voting_list[2]['vote'] = generate_random_strings(30, [0 ,1, 2, 3, 4])
# voting_list[3]['vote'] = generate_random_strings(40, [0 ,1, 2, 3, 4])
# voting_list[4]['vote'] = generate_random_strings(50, [0 ,1, 2, 3, 4])

for election in voting_list:
    decentral_elect.start_vote(election['id'])
    print('Status:', decentral_elect.get_status(election['id']))

    pkeys = decentral_elect.get_registered_pkeys(election['id'])
    pkeys = pkeys_type_transform(pkeys)
    for ballot, voter in zip(election['vote'], election['registers']):
        message = hash_vote(ballot)
        signature = uaosring_sign(pkeys, (voter.public_key, voter.private_key), message)

        linkable_ring_signature = LinkableRingSignature(
            pkeys = pkeys, 
            tag = signature[1], 
            ring = signature[2], 
            seed = signature[3], 
            message = message
            )

        decentral_elect.vote(election['id'], linkable_ring_signature)

        if (decentral_elect.check_if_voted(election['id'], linkable_ring_signature.tag[0].n)):
            print('Voted')
        else:
            print("Unvoted")


Status: Vote
Voted
Voted
Voted
Voted
Voted
Voted
Voted
Voted
Voted
Voted


In [7]:
message = DecentralElectContract.hash_vote('2_apple')
v = voting_list[0]['registers'][6]
identifier = 0

pkeys = decentral_elect.get_registered_pkeys(identifier)
pkeys = pkeys_type_transform(pkeys)

signature = uaosring_sign(pkeys, (v.public_key, v.private_key), message)
signature[2][0] += 3

linkable_ring_signature = LinkableRingSignature(
    pkeys = pkeys, 
    tag = signature[1], 
    ring = signature[2], 
    seed = signature[3], 
    message = message
    )

decentral_elect.vote(1, linkable_ring_signature)

if (decentral_elect.check_if_voted(identifier, linkable_ring_signature.tag[0].n)):
    print('Voted')
else:
    print("UnVoted")


Voted


In [8]:
# verify ballot

result_list = voting_list

for election in result_list:
    decentral_elect.stop_vote(election['id'])

    # ballots = decentral_elect.get_ballots(election['id'])

    for ballot in election['vote']:
        decentral_elect.verify_ballot(election['id'], ballot)

In [9]:
# Show result

for election in result_list:
    candidates = decentral_elect.get_candidates(election['id'])
    for candidate in candidates:
        print(candidate)

(0, 'Apple', 'Red', 2)
(1, 'Grape', 'Purple', 1)
(2, 'Orange', 'Orange', 2)
(3, 'Guava', 'Green', 3)
(4, 'Banana', 'Yellow', 0)


In [10]:
print(decentral_elect.total_gas_spend)

14001763
